<a href="https://colab.research.google.com/github/jorgeriva/MIAR-Algoritmos-Optimizacion/blob/main/2_Convocatoria_de_Jorge_Rivadulla_Brey_Trabajo_Pr%C3%A1ctico_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Jorge Rivadulla Brey  <br>
Url: https://github.com/jorgeriva/MIAR-Algoritmos-Optimizacion/blob/main/2_Convocatoria_de_Jorge_Rivadulla_Brey_Trabajo_Pr%C3%A1ctico_Algoritmos.ipynb <br>
Google Colab:https://colab.research.google.com/drive/1_-YMfYo9ItIoyf3H-KBmdmFV0bFfe0OG?usp=sharing  <br>

Problema:
Organizar los horarios de partidos de una jornada de La Liga<br>

Descripción del problema:

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de
liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un
algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la
audiencia.

....







                                        

#Modelo
- ¿Como represento el espacio de soluciones?


El espacio de soluciones se encuentra definido por todas las posibles combinaciones que hay para colocar los diez partidos que forman la jornada en sus respecitivos horarios. Es decir, se trata de todas las posibles formas de definir una jornada con los partidos ya decididos mientras mantenemos que se cumplan unas restricciones.

Cada posible solución dentro de este espacio puede representarse mediante una matriz, donde:

*   Las filas corresponden a los diferentes partidos que deben disputarse en la jornada
*   Las columnas representan los distintos horarios disponibles para la programación de los encuentros.


  En esta matriz, cada partido debe asignarse exclusivamente a un único horario, lo que implica que cada fila debe contener exactamente una selección válida dentro del conjunto de horarios disponibles. Este enfoque en forma de matriz facilita la visualización de una configuración particular de la jornada como la aplicación distintos algoritmos para obtener la solución óptima.

  Dado que el número de combinaciones es muy elevado el espacio de soluciones será de gran tamaño.

- ¿Cual es la función objetivo?

  El objetivo del problema es maximizar el valor de la jornada, de manera que los partidos que van a provocar una mayor audiencia se jueguen en los mejores horarios. Esto significa que la planificación de la jornada debe priorizar la asignación de encuentros de alto impacto en franjas horarias que garanticen una mayor visibilidad y, en consecuencia, una mayor audiencia global.

  Sabiendo esto, la función objetivo debe ser la maximización del sumatorio del producto del valor de los partidos de la jornada por sus horarios. Es decir, la función que nos dé como resultado la ordenación de los partidos que nos genera la jornada con una mayor audiencia posible, la cual se expresa de la siguiente forma:

$$
\max \sum_{i=1}^{N} V_i \cdot H_i
$$

* $V_i$ representa el valor asignado al partido $i$, determinado por el valor de la audiencia del partido por la correlacion si hay más partidos en ese horario.  

* $H_i$ es el coeficiente asociado al horario en el que se programa el partido $i$, reflejando el valor que tien cada uno de esos horarios.  

La sumatoria recorre todos los partidos programados en la jornada.



- ¿Como implemento las restricciones?

Contamos con tres restricciones, las cuales comentaremos aquí e implementaremos en el espacio para código que tenemos justo debajo.

1. Cada equipo solo puede jugar un partido. Con esto evitamos que un equipo aparezca en más de un espacio de la tabla de la jornada.
2. Solo se puede jugar en los horarios marcados. Para evitar que algún partido se salga de los espacios de la tabla.
3. Penalización por coincidencias: Si varios partidos coinciden en horario, su audiencia se reduce siguiendo las ponderaciones de la tabla que veremos más adelante.




In [20]:
import numpy as np
import random
import pandas as pd

# Datos del problema
Partido_AA = 2
Partido_AB = 1.3
Partido_AC = 1
Partido_BB = 0.90
Partido_BC = 0.75
Partido_CC = 0.47

matches = {
    ('Celta', 'Real Madrid'): Partido_AB,
    ('Valencia', 'R. Sociedad'): Partido_AB,
    ('Mallorca', 'Eibar'): Partido_CC,
    ('Athletic', 'Barcelona'): Partido_AB,
    ('Leganes', 'Osasuna'): Partido_CC,
    ('Villarreal', 'Granada'): Partido_BC,
    ('Alaves', 'Levante'): Partido_BB,
    ('Espanyol', 'Sevilla'): Partido_BB,
    ('Betis', 'Valladolid'): Partido_BC,
    ('Atletico', 'Getafe'): Partido_BB,
}

horarios = ["Sábado 20", "Domingo 20", "Domingo 18", "Sábado 18", "Domingo 16", "Sábado 16", "Sábado 12", "Domingo 12", "Viernes 20", "Lunes 20"]
coef_horario = {"Viernes 20": 0.4, "Sábado 12": 0.55, "Sábado 16": 0.7, "Sábado 18": 0.8, "Sábado 20": 1,
                "Domingo 12": 0.45, "Domingo 16": 0.75, "Domingo 18": 0.85, "Domingo 20": 1, "Lunes 20": 0.4}

coef_coincidencia = {1: 1, 2: 0.75, 3: 0.65, 4: 0.4, 5: 0.30, 6: 0.25, 7: 0.22, 8: 0.20, 9: 0.20}

# Asignación de horarios con restricciones
def aplicar_restricciones():
    asignacion = {}
    disponibles = list(horarios)
    partidos = list(matches.keys())
    random.shuffle(partidos)

    # Asegurar al menos un partido el viernes y otro el lunes
    partido_viernes = partidos.pop()
    asignacion[partido_viernes] = "Viernes 20"

    partido_lunes = partidos.pop()
    asignacion[partido_lunes] = "Lunes 20"

    # Asignar los demás partidos libremente
    for partido in partidos:
        horario = random.choice(disponibles)
        asignacion[partido] = horario

    return asignacion

# Calcular audiencia total considerando penalizaciones
def calcular_audiencia_total(asignacion):
    penalizacion_coincidencia = {}
    for horario in asignacion.values():
        penalizacion_coincidencia[horario] = penalizacion_coincidencia.get(horario, 0) + 1

    audiencia_total = 0
    for partido, horario in asignacion.items():
        audiencia_base = matches[partido]
        coincidencias = penalizacion_coincidencia[horario]
        penalizacion = coef_coincidencia.get(coincidencias, 1)
        audiencia_total += audiencia_base * coef_horario[horario] * penalizacion

    return audiencia_total

# Ejecución
asignacion = aplicar_restricciones()
audiencia_total = calcular_audiencia_total(asignacion)

# Mostrar asignación de partidos
print("\nAsignación de partidos y horarios:")
for partido, horario in asignacion.items():
    print(f"{partido[0]} vs {partido[1]} -> {horario}")

print(f"\nAudiencia total de la jornada: {audiencia_total:.2f} millones de espectadores")



Asignación de partidos y horarios:
Atletico vs Getafe -> Viernes 20
Valencia vs R. Sociedad -> Lunes 20
Athletic vs Barcelona -> Domingo 20
Betis vs Valladolid -> Domingo 18
Espanyol vs Sevilla -> Lunes 20
Mallorca vs Eibar -> Domingo 12
Leganes vs Osasuna -> Sábado 12
Alaves vs Levante -> Viernes 20
Celta vs Real Madrid -> Lunes 20
Villarreal vs Granada -> Domingo 16

Audiencia total de la jornada: 4.42 millones de espectadores


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [21]:
import math

# Calcular las combinaciones C(19, 10) y C(17, 8)
C_19_10 = math.comb(19, 10)
C_17_8 = math.comb(17, 8)

# Calcular la cantidad total de combinaciones
total_combinations = C_19_10 - C_17_8
total_combinations

print(f"El número de jornadas posibles es : {total_combinations}")






El número de jornadas posibles es : 68068


El problema es una variación del problema de asignación donde el objetivo es maximizar la audiencia total. Tambien podemos ver que este problema es una mezcal entre el problema de asiganción de tareas y el problema de asigancion cuadrática, sabiendo esto podemos decir que es un problema NP-hard, por lo que encontrar una solución optima es dificil. Por lo tanto nos parece un buena solución el implementar un algoritmo voraz para resolverlo.

Este algoritmo esta formado por los siguientes apartados:
- Ordenación: O(N² log N) (colocación de los partidos en las franjas que provoquen la mayor audiencia)
- Bucle principal: O(N²)
- Cálculo de audiencia total: O(1)

por lo cual su complejidad es O(N² log N)


Para contabilizar el espacio de soluciones debemos resolver un problema de combinatoria con la intención de ver cuantas posibles jornadas podrían llegar a producirse teniendo en cuenta que ya conocemos los enfrentamientos que van a realizarse.

Para representar ese problema nos lo hemos imaginado como un problema de meter pelotas en cajas numeradas. Para calcular la cantidad de combinaciones posibles de distribuir los 10 objetos en 10 cajas numeradas, con la restricción de que las cajas 1 y 2 no pueden quedar vacías (Son las que equivalen a viernes y a lunes), podemos abordar el problema paso a paso:

Si no tuviéramos ninguna restricción, el problema sería simplemente una distribución de 10 objetos en 10 cajas, con la posibilidad de que una caja contenga más de un objeto. Este es un problema clásico de combinaciones con repetición, que se puede calcular mediante la fórmula de combinaciones con repetición:

Lo primero será calcular cuantas posibles combinaciones de partidos hay para vierens y lunes, que lo haremos por combinación simple:

$$ \
C(n + k - 1, k)
\ $$

donde \(n\) es el número de objetos (10) y \(k\) es el número de cajas (10). En este caso, la fórmula sería:

$$\
C(10 + 10 - 1, 10) = C(19, 10)
\$$

Pero si que contamos con restricciones, así que para encontrar los casos en los que la combinación no tiene las cajas correspondientes a lunes y viernes vacias debemos debemos restar al calculo anterior las distribuciones en las cuales las cajas 1 y 2 están vacías. Si las cajas 1 y 2 están vacías, entonces solo tenemos que distribuir los 10 objetos en las 8 cajas restantes (del 3 al 10). La cantidad de formas de hacer esta distribución es:

$$\
C(10 + 8 - 1, 8) = C(17, 10)
\$$

Por lo tanto, la cantidad total de combinaciones que satisface la condición de que las cajas 1 y 2 no estén vacías es:

$$\
C(19, 10) - C(17, 8)
\$$

Esto traducido a nuestro problema es restar todas las combinaciones de jornadas posibles menos aquellas en las que no hay ningún partido ni el lunes ni el viernes.
### Cálculo de las combinaciones

$$\
C(19, 10) = \frac{19!}{10!(19-10)!} = \frac{19!}{10!9!}
\$$

$$\
C(17, 8) = \frac{17!}{8!(17-8)!} = \frac{17!}{8!9!}
\$$

Finalmente, el total de combinaciones es:

$$\
C(19, 10) - C(17, 8) = 68068
\$$

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

In [60]:
import numpy as np
import random
import pandas as pd

# Datos del problema
Partido_AA = 2
Partido_AB = 1.3
Partido_AC = 1
Partido_BB = 0.90
Partido_BC = 0.75
Partido_CC = 0.47

team_categories = {
    'Celta': 2, 'Real Madrid': 1, 'Valencia': 2, 'R. Sociedad': 1, 'Mallorca': 3,
    'Eibar': 3, 'Athletic': 2, 'Barcelona': 1, 'Leganes': 3, 'Osasuna': 3,
    'Villarreal': 2, 'Granada': 3, 'Alaves': 2, 'Levante': 2, 'Espanyol': 2,
    'Sevilla': 2, 'Betis': 2, 'Valladolid': 3, 'Atletico': 2, 'Getafe': 2
}

horarios = ["Sábado 20", "Domingo 20", "Domingo 18", "Sábado 18", "Domingo 16", "Sábado 16", "Sábado 12", "Domingo 12", "Viernes 20", "Lunes 20"]
coef_horario = {"Viernes 20": 0.4, "Sábado 12": 0.55, "Sábado 16": 0.7, "Sábado 18": 0.8, "Sábado 20": 1,
                "Domingo 12": 0.45, "Domingo 16": 0.75, "Domingo 18": 0.85, "Domingo 20": 1, "Lunes 20": 0.4}

# Coeficiente de penalización por coincidencia
coef_coincidencia = {1: 1, 2: 0.75, 3: 0.65, 4: 0.4, 5: 0.30, 6: 0.25, 7: 0.22, 8: 0.20, 9: 0.20}

# Enfrentamientos fijos
matches = {
    ('Celta', 'Real Madrid'): Partido_AB,
    ('Valencia', 'R. Sociedad'): Partido_AB,
    ('Mallorca', 'Eibar'): Partido_CC,
    ('Athletic', 'Barcelona'): Partido_AB,
    ('Leganes', 'Osasuna'): Partido_CC,
    ('Villarreal', 'Granada'): Partido_BC,
    ('Alaves', 'Levante'): Partido_BB,
    ('Espanyol', 'Sevilla'): Partido_BB,
    ('Betis', 'Valladolid'): Partido_BC,
    ('Atletico', 'Getafe'): Partido_BB,
}

# Función para calcular la audiencia de un partido específico
def calcular_audiencia_partido(equipo1, equipo2, horario, asignacion):
    nivel1, nivel2 = team_categories[equipo1], team_categories[equipo2]

    if nivel1 == 1 and nivel2 == 1:
        audiencia_base = Partido_AA
    elif (nivel1 == 1 and nivel2 == 2) or (nivel1 == 2 and nivel2 == 1):
        audiencia_base = Partido_AB
    elif (nivel1 == 1 and nivel2 == 3) or (nivel1 == 3 and nivel2 == 1):
        audiencia_base = Partido_AC
    elif nivel1 == 2 and nivel2 == 2:
        audiencia_base = Partido_BB
    elif (nivel1 == 2 and nivel2 == 3) or (nivel1 == 3 and nivel2 == 2):
        audiencia_base = Partido_BC
    elif nivel1 == 3 and nivel2 == 3:
        audiencia_base = Partido_CC

    # Penalización por coincidencias
    penalizacion_coincidencia = {}
    for h in asignacion.values():
        penalizacion_coincidencia[h] = penalizacion_coincidencia.get(h, 0) + 1
    coincidencias = penalizacion_coincidencia.get(horario, 1)
    penalizacion = coef_coincidencia.get(coincidencias, 1)

    return audiencia_base * coef_horario[horario] * penalizacion

# Función para calcular la audiencia total
def calcular_audiencia_total(asignacion):
    return sum(calcular_audiencia_partido(p[0], p[1], h, asignacion) for p, h in asignacion.items())

# Algoritmo voraz para asignar horarios a los partidos
def greedyalgorithm():
    partidos_posibles = list(matches.keys())  # Usar los enfrentamientos proporcionados

    asignacion = {}
    disponibles = list(horarios)

    asignado_viernes = False
    asignado_lunes = False

    # Asignar partidos a horarios, permitiendo coincidencias en el mismo horario
    for horario in horarios:
        mejor_partido = None
        mejor_audiencia = 0

        for partido in partidos_posibles:
            if partido[0] not in asignacion and partido[1] not in asignacion:
                audiencia = calcular_audiencia_partido(partido[0], partido[1], horario, asignacion)
                if audiencia > mejor_audiencia:
                    mejor_partido = partido
                    mejor_audiencia = audiencia

        if mejor_partido:
            asignacion[mejor_partido] = horario
            partidos_posibles.remove(mejor_partido)

            # Marcar los horarios de viernes y lunes si asignamos un partido a esos horarios
            if horario == "Viernes 20":
                asignado_viernes = True
            if horario == "Lunes 20":
                asignado_lunes = True

    # Si no se asignó un partido al viernes, asignamos uno
    if not asignado_viernes:
        for partido in partidos_posibles:
                asignacion[partido] = "Viernes 20"
                break

    # Si no se asignó un partido al lunes, asignamos uno
    if not asignado_lunes:
        for partido in partidos_posibles:
                asignacion[partido] = "Lunes 20"
                break

    return asignacion


# Ejecutar el algoritmo voraz
asignacion_optima = greedyalgorithm()
audiencia_total = calcular_audiencia_total(asignacion_optima)

# Mostrar los resultados
print("\nJornada óptima generada con algoritmo voraz:")
for partido, horario in asignacion_optima.items():
    print(f"{partido[0]} vs {partido[1]} -> {horario}")
print(f"\nAudiencia total de la jornada: {audiencia_total:.2f} millones de espectadores")



Jornada óptima generada con algoritmo voraz:
Celta vs Real Madrid -> Sábado 20
Valencia vs R. Sociedad -> Domingo 20
Athletic vs Barcelona -> Domingo 18
Alaves vs Levante -> Sábado 18
Espanyol vs Sevilla -> Domingo 16
Atletico vs Getafe -> Sábado 16
Villarreal vs Granada -> Sábado 12
Betis vs Valladolid -> Domingo 12
Mallorca vs Eibar -> Viernes 20
Leganes vs Osasuna -> Lunes 20

Audiencia total de la jornada: 6.86 millones de espectadores


Elección del algoritmo voraz:

Para resolver este problema de asignación de horarios a partidos, decidimos utilizar un algoritmo voraz debido a su simplicidad y eficiencia, así como a su adecuación para este tipo de problema. Un algoritmo voraz toma decisiones paso a paso, eligiendo la opción que parece más prometedora en cada etapa, sin considerar el impacto global de esas decisiones. En este caso, el algoritmo asigna los partidos a los horarios de mayor audiencia, buscando maximizar el número total de espectadores.

Justificación de la elección:

Eficiencia en tiempo y complejidad: Los algoritmos voraces son conocidos por ser rápidos, lo que los hace ideales cuando se tiene un espacio de datos pequeño y cuando la solución óptima no es estrictamente necesaria. Este es el caso en nuestro problema, ya que el número de partidos es limitado (solo diez), lo que permite que el algoritmo funcione de manera eficiente sin generar una complejidad computacional alta.

Simplicidad de implementación: El algoritmo voraz es sencillo de implementar en comparación con otros métodos más complejos, como los algoritmos de búsqueda exhaustiva o técnicas de optimización combinatoria. Este aspecto de simplicidad reduce los riesgos de errores y facilita el proceso de desarrollo.

Maximización de la audiencia: El objetivo principal del problema es maximizar la audiencia, y el algoritmo voraz puede hacerlo de manera efectiva al asignar a cada horario el partido con la mayor audiencia posible en ese momento. Este enfoque es adecuado porque no necesitamos considerar todas las combinaciones posibles, sino que podemos ir asignando los partidos de manera directa, optimizando el resultado en cada paso.

Dentro de los algoritmos voraces hemos implementado una variación del algorítmo de selección de actividades (Interval Scheduling). El problema de selección de actividades consiste en escoger el mayor número posible de actividades que no se solapen en el tiempo. Se utiliza un algoritmo voraz que selecciona la actividad que termina más temprano para dejar la mayor cantidad de espacio libre en el futuro. En este caso tenemos los tiempos determinados por los horarios de la jornada.

Nuestro problema de asignación de horarios a partidos tiene similitudes con la selección de actividades, ya que:

Cada partido es una actividad que debe asignarse a un horario sin superposición de equipos. Es decir un partido no puede jugarse en dos horarios distintos.

Queremos maximizar la audiencia total mientras cumplimos las restricciones de horarios y equipos. Además de que aplicaremos penalizaciones si dos equipos juegan a la vez.

Nuestro código es una variación del algoritmo de selección de actividades, pero en lugar de minimizar solapamientos de tiempo, evitamos que los equipos jueguen en horarios repetidos si con eso podemoss maximizar la audiencia.

No ordenamos las actividades por tiempo de finalización, sino que iteramos sobre los horarios y buscamos el mejor partido posible para cada uno.

En lugar de seleccionar la actividad que termina más pronto, seleccionamos el partido con mayor audiencia posible para cada horario, buscando generar la mayor audiencia posible.

Agregamos restricciones adicionales para asegurarnos de que al menos haya un partido en viernes y en lunes, lo cual no existe en el problema clásico de selección de actividades.


El algoritmo voraz de selección de actividades es una base sólida para nuestra solución porque nos permite asignar partidos de manera eficiente sin necesidad de probar todas las combinaciones posibles. Las modificaciones que hicimos lo adaptan a las necesidades del problema, priorizando la audiencia y asegurando restricciones de equipos y horarios.

Posibles mejoras:

Aunque el algoritmo voraz es eficiente y proporciona una solución adecuada, no garantiza una solución óptima global. Una posible mejora sería utilizar un enfoque híbrido que combine la estrategia voraz con técnicas como la búsqueda local o el algoritmo de listas tabú para explorar soluciones cercanas y encontrar una mejor asignación.

Otra opción sería utilizar programación dinámica o backtracking, los cuales podrían encontrar la mejor solución global, pero a costa de una mayor complejidad computacional y tiempos de ejecución más largos.

Conclusión:

El algoritmo voraz ha sido una elección adecuada para este problema debido a su simplicidad, eficiencia y capacidad para generar una solución cercana a la óptima en tiempos razonables. Si bien hay posibilidades de mejora con enfoques más complejos, el algoritmo actual cumple con los requisitos del problema de manera efectiva.